In [1]:
from qiskit import QuantumCircuit

from qiskit_aer import AerSimulator, noise
from copy import copy

import numpy as np

def get_noise_model(p):
    p_1, p_2, p_m = p,p,p
    e_1 = noise.depolarizing_error(p_1, 1)
    e_2 = noise.depolarizing_error(p_2, 2)
    e_m = noise.pauli_error([('X',p_m),('I',1-p_m)])
    noise_model = noise.NoiseModel()
    noise_model.add_all_qubit_quantum_error(e_1, ['h', 'x', 'rz', 'sx'])
    noise_model.add_all_qubit_quantum_error(e_2, ['cx'])
    noise_model.add_all_qubit_quantum_error(e_m, ['measure'])
    return noise_model

import sys
sys.path.append("../")
from devices import supportedDevices, getLayout
from QuantumAwesomeness import *

In [17]:
device = 'ibmqx2'
q = 0.2

backend = AerSimulator()
noise_model = get_noise_model(q)

In [18]:
results = []
for s in range(500):
    print(s)
    with open('../results/'+device+'/qasm/qasm_C_8192_'+str(s)+'.txt', 'r') as file:
        qasms = file.readlines()[1::]

    qcs = []
    for qasm in qasms:
        qc = QuantumCircuit()
        qc = qc.from_qasm_str(qasm.replace('\\n','\n'))
        qcs.append(qc)

    job = backend.run(qcs, shots=8192, noise_model=noise_model)

    result = []
    for result_raw in job.result().get_counts():
        result.append({})
        for string, nums in result_raw.items():
            result[-1][string[::-1]] = nums
    results.append(result)

file = open('../results/'+device+'/results_move=C_shots=8192_sim=True.txt', 'a')
for result in results:
    file.write(str(result))
file.close()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [19]:
num, area, entangleType, pairs, pos, example, sdk, runs = getLayout(device)

file_one = open('../results/'+device+'/oneProbs_move=C_shots=8192_sim=True.txt', 'a')
file_same = open('../results/'+device+'/sameProbs_move=C_shots=8192_sim=True.txt', 'a')
for r, result in enumerate(results):
    oneProbs = []
    sameProbs = []
    for resultsRaw in result:
        oneProb, sameProb, _ = processResults ( dict(resultsRaw), num, pairs, False, 8192 )
        oneProbs.append(oneProb)
        sameProbs.append(sameProb)
    file_one.write((r!=0)*'\n'+str(oneProbs))
    file_same.write((r!=0)*'\n'+str(sameProbs))
file_one.close()
file_same.close()

In [79]:
s = 2

invpairs = {}
for name, qs in pairs.items():
    invpairs[tuple(qs)] = name
    invpairs[tuple(qs[::-1])] = name

with open('../results/oldEagle/gates_move=C_shots=8192_sim=True.txt', 'r') as file:
    gates = [eval(line) for line in file.readlines()]

with open('../results/oldEagle/qasm/qasm_C_8192_'+str(s)+'.txt', 'r') as file:
    qasms = [line for line in file.readlines()][1::]


qasm_pairs = set()
for line in qasms[t].split('\\n'):
    if 'cx' in line:
        con = line[line.find('[')+1:line.find(']')]
        line2 = line[line.find(']')+1::]
        tar = line2[line2.find('[')+1:line2.find(']')]
        qs = (int(con),int(tar))
        pair = invpairs[qs]
        qasm_pairs.add(pair)

gates[s][0].keys() == qasm_pairs

True

In [87]:
len(gates[0])

20